# "[SSUDA] Pytorch로 LSTM, Prophet 사용하기 with 태양광 발전량 예측 경진대회"
- author: Seong Yeon Kim 
- categories: [DACON, jupyter, SSUDA, Pytorch, Deep Learning, LSTM]

# LSTM

# 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

path = '/content/drive/MyDrive/sun/'

energy = pd.read_csv(path + 'energy.csv')
sample_submission = pd.read_csv(path + 'sample_submission.csv')
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25632 entries, 0 to 25631
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   time               25632 non-null  object 
 1   dangjin_floating   25608 non-null  float64
 2   dangjin_warehouse  25584 non-null  float64
 3   dangjin            25632 non-null  int64  
 4   ulsan              25632 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 1001.4+ KB


In [3]:
energy.fillna(energy.mean(),inplace = True)
energy = energy.set_index('time')
energy.head()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan
time,,,,
2018-03-01 1:00:00,0.0,0.0,0,0
2018-03-01 2:00:00,0.0,0.0,0,0
2018-03-01 3:00:00,0.0,0.0,0,0
2018-03-01 4:00:00,0.0,0.0,0,0
2018-03-01 5:00:00,0.0,0.0,0,0


결측치를 채우고 시간을 인덱스로 바꿨습니다.

In [11]:
ulsan = energy['ulsan'].values.astype(float)
dangjin_floating = energy['dangjin_floating'].values.astype(float)
dangjin_warehouse = energy['dangjin_warehouse'].values.astype(float)
dangjin	 = energy['dangjin'].values.astype(float)

데이터를 각각 뽑아냅니다.

# dangjin_floating

In [12]:
# hyper parameters

learning_rate = 0.0001 
sequence_length = 12 # 24 일때가 가장 좋았음. but 코렙 램이 터지는 관계로 12 사용
epochs = 2000

In [13]:
# 데이터를 리스트 형태로 변경

def make_batch(input_data, sl):
    train_x = []
    train_y = []
    
    L = len(input_data)

    for i in range(L-sl):
        # sl기간 만큼 있는 데이터에서 다음 시점 맞추기.
        train_seq = input_data[i:i+sl]
        train_label = input_data[i+sl:(i+sl+1)]
        # 리스트 값을 train_x에 어팬드함.
        train_x.append(train_seq)
        train_y.append(train_label)
    
    return train_x, train_y

In [14]:
class simple_lstm(nn.Module):

    def __init__(self):
        super().__init__()
        self.input_vector = 1 # 입력 벡터 길이
        self.sequence_length = 12 # 데이터 묶음 길이(24개 데이터 사용)
        self.output_vector = 100 # 은닉층 사이즈
        self.num_layers = 4 # 층 개수

        self.lstm = nn.LSTM(input_size = self.input_vector, hidden_size = self.output_vector,
                            num_layers = self.num_layers, batch_first = True)
        self.linear = nn.Sequential(
            nn.Linear(self.output_vector, 50),
            nn.Linear(50, 30),
            nn.Linear(30, 10),
            nn.Linear(10, 1)
        )

    def forward(self, x):
        output, _ = self.lstm(x)
        return self.linear(output[:, -1, :])

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [15]:
train_x, train_y = make_batch(dangjin_floating.reshape(-1,1), sequence_length)

# 텐서에 데이터 실기
tensor_x = torch.Tensor(train_x)
tensor_y = torch.Tensor(train_y)

dangjin_floatings = simple_lstm()
# 모델을 디바이스에 실음 (디바이스에는 모델과 데이터를 실어야함)
dangjin_floatings = dangjin_floatings.to(device)

# 아담 옵티마이저 사용 (학습하려는 모델 파라미터, 학습률)
optimizer = torch.optim.Adam(dangjin_floatings.parameters(), lr = learning_rate)
criterion = nn.MSELoss()


for i in range(epochs):
    # 모델 학습 모드
    dangjin_floatings.train()
    tensor_x = tensor_x.to(device)
    tensor_y = tensor_y.to(device)
    output = dangjin_floatings(tensor_x)
    loss = criterion(output, tensor_y.view(-1,1))

    # 옵티마이저 초기화 (배치마다 해줘야함)
    optimizer.zero_grad()
    # 로스함수를 사용해 역전파
    loss.backward()
    # 옵티마이저를 이용해 가중치 업데이트
    optimizer.step()

    # 100번째 배치마다 로스 값 출력
    if i % 100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))



Epoch 0, Loss 51745.66797
Epoch 100, Loss 51092.85156
Epoch 200, Loss 47514.81641
Epoch 300, Loss 41005.27734
Epoch 400, Loss 37112.99609
Epoch 500, Loss 36839.90625
Epoch 600, Loss 36838.98828
Epoch 700, Loss 36828.25000
Epoch 800, Loss 15298.88477
Epoch 900, Loss 4058.15771
Epoch 1000, Loss 2399.74683
Epoch 1100, Loss 2103.90039
Epoch 1200, Loss 2016.97937
Epoch 1300, Loss 1976.83508
Epoch 1400, Loss 1952.15393
Epoch 1500, Loss 1933.21155
Epoch 1600, Loss 1917.11658
Epoch 1700, Loss 1901.57031
Epoch 1800, Loss 1888.42883
Epoch 1900, Loss 1876.48096


In [16]:
x_input = np.array(energy.dangjin_floating[-12:])
x_input = x_input.reshape((1,12,1))
dangjin_floating_pred = []

for i in range(672):
    
    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)

    # 모델에 넣고 output값 확인 위해서는 cpu로 돌린 뒤 넘파이로 변환.
    predict = dangjin_floatings(x_input).cpu().detach().numpy()
    # 예측값 배출
    new_input = predict.reshape((1,1,1))

    # 예측값을 실제값인 것 처럼 재사용하여 다시 모델에 넣음(나이브한 방식)
    x_input = np.concatenate((x_input[:,-11:].cpu(), new_input), axis = 1)

    # 예측값은 dangjin_floating_pred 리스트에 계속 저장해둠
    dangjin_floating_pred.append(predict[0][0])

# dangjin_warehouse

In [22]:
# 함수 사용해 데이터 정해진 길이만큼 짜르고 y값 만들고 하기
# train_x = [길이 12 어레이, 길이 12 어레이, ...]
# train_y = [길이 1 어레이, 길이 1 어레이, ..]
train_x, train_y = make_batch(dangjin_warehouse.reshape(-1,1), sequence_length)

# 데이터 텐서로 실기(리스트도 실기 가능)
tensor_x = torch.Tensor(train_x)
tensor_y = torch.Tensor(train_y)

# 모델 불러와서 디바이스에 실기 (GPU 사용 위해선 모델과 데이터를 실어야함)
dangjin_warehouses = simple_lstm()
dangjin_warehouses = dangjin_warehouses.to(device)

# 옵티마이저 아담(많이 사용됨) 사용
optimizer = torch.optim.Adam(dangjin_warehouses.parameters(), lr = learning_rate)

# 손실함수 MSE 사용
criterion = nn.MSELoss()

for i in range(epochs):
    # 학습 모드(가중치 업데이트 됨)로 변환
    dangjin_warehouses.train()

    # 데이터(텐서 형식) 디바이스에 실기
    tensor_x = tensor_x.to(device)
    tensor_y = tensor_y.to(device)

    # 모델에 x 데이터 넣기
    output = dangjin_warehouses(tensor_x)

    # 로스 값 구하기
    loss = criterion(output, tensor_y.view(-1,1))

    # 옵티마이저 초기화(매 배치마다 초기화 해야함)
    optimizer.zero_grad()
    # 손실함수 이용해 역전파
    loss.backward()
    # 옵티마이저 사용해 가중치 업데이트
    optimizer.step()

    # 100번째 배치마다 로스 계산
    if i % 100 == 0:
        # loss.item 함수 사용하면 로스 값 제출해줌
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))

# 테스트 데이터 계산 위해 맨 뒤 12개 데이터 사용
x_input = np.array(energy.dangjin_warehouse[-12:])
x_input = x_input.reshape((1,12,1))
dangjin_warehouse_pred = []

Epoch 0, Loss 29900.15234
Epoch 100, Loss 29317.77930
Epoch 200, Loss 26276.70508
Epoch 300, Loss 22257.02539
Epoch 400, Loss 21406.61328
Epoch 500, Loss 21391.32422
Epoch 600, Loss 8447.42188
Epoch 700, Loss 2214.76782
Epoch 800, Loss 1341.09875
Epoch 900, Loss 1182.24084
Epoch 1000, Loss 1131.52197
Epoch 1100, Loss 1107.95605
Epoch 1200, Loss 1092.98474
Epoch 1300, Loss 1083.00134
Epoch 1400, Loss 1074.64624
Epoch 1500, Loss 1068.63171
Epoch 1600, Loss 1064.08704
Epoch 1700, Loss 1058.59131
Epoch 1800, Loss 1054.51086
Epoch 1900, Loss 1050.88635


In [18]:
for i in range(672):
    
    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = dangjin_warehouses(x_input).cpu().detach().numpy()

    new_input = predict.reshape((1,1,1))
    x_input = np.concatenate((x_input[:,-23:].cpu(), new_input), axis = 1)
    dangjin_warehouse_pred.append(predict[0][0])

# dangjin

# ulsan

나머지 2개 변수 예측은 추후에 lstm 코드 복습 시 사용. (앞 두 변수 방식과 동일)

# 결과 제출

In [ ]:
sample_submission.iloc[:24*28, 1] = dangjin_floating_pred
sample_submission.iloc[:24*28, 2] = dangjin_warehouse_pred
# sample_submission.iloc[:24*28, 3] = dangjin_pred
# sample_submission.iloc[:24*28, 4] = ulsan_pred
# sample_submission

# Prophet

# 필요한 패키지 설치하기

In [19]:
#!pip install neuralprophet
from datetime import datetime
from neuralprophet import NeuralProphet

energy = pd.read_csv(path + 'energy.csv')
sample_submission = pd.read_csv(path + 'sample_submission.csv')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 74 kB 3.7 MB/s 


In [20]:
# 시간 1 ~ 24 => 0 ~ 23 으로 변환
def convert_time(x):
    # 2018-03-01 1:00:00 값 변환하기
    Ymd, HMS = x.split(' ')
    H, M, S = HMS.split(':')
    H = str(int(H) - 1)

    # 다시 시간 합치기
    HMS = ':'.join([H, M, S])
    return ' '.join([Ymd, HMS])

energy['time'] = energy['time'].apply(lambda x:convert_time(x))

# 모델 적용하기

In [23]:
column = 'dangjin_floating'

df = pd.DataFrame()
df['ds'] = energy['time']
df['y'] = energy[column]

In [24]:
# 모델 설정
model = NeuralProphet()

# 훈련
loss = model.fit(df, freq = 'H')

# 미래 예측용 데이터 프레임 만들기
df_pred = model.make_future_dataframe(df, periods = 18000)

# 미레 예측 하기
predict = model.predict(df_pred)

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.996% of the data.
INFO:NP.df_utils:Major frequency H corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO:NP.df_utils:Defined frequency is equal to major frequency - H
INFO - (NP.forecaster.__handle_missing_data) - dropped 24 NAN row in 'y'
INFO:NP.forecaster:dropped 24 NAN row in 'y'
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO:NP.config:Setting normalization to global as only one dataframe provided for training.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
INFO:NP.config:Auto-set batch_size to 64
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 81
INFO:NP.config:Auto-set epochs to 81


  0%|          | 0/160 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.76E-02, min: 1.19E+00
INFO:NP.utils_torch:lr-range-test results: steep: 6.76E-02, min: 1.19E+00


  0%|          | 0/160 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.76E-02, min: 1.19E+00
INFO:NP.utils_torch:lr-range-test results: steep: 6.76E-02, min: 1.19E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 7.71E-02
INFO:NP.forecaster:lr-range-test selected learning rate: 7.71E-02
Epoch[81/81]: 100%|██████████| 81/81 [01:16<00:00,  1.06it/s, SmoothL1Loss=0.0161, MAE=67.5, RMSE=101, RegLoss=0]
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.996% of the data.
INFO:NP.df_utils:Major frequency H corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO:NP.df_utils:Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.994% of the data.
INFO:NP.df_utils:Major frequency H corresponds to 99.994% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to majo

In [25]:
predict_1 = predict.copy()
predict_1 = predict_1.query('ds >= "2021-02-01 00:00:00"')
predict_1 = predict_1.query('ds < "2021-03-01 00:00:00"')

# 2021-06-09 ~ 2021-07-09
predict_2 = predict.copy()
predict_2 = predict_2.query('ds >= "2021-06-09 00:00:00"')
predict_2 = predict_2.query('ds < "2021-07-09 00:00:00"')

# 제출 파일 업데이트
sample_submission[column] = list(predict_1['yhat1']) + list(predict_2['yhat1'])
sample_submission.head()

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,-29.331793,0,0,0
1,2021-02-01 02:00:00,-28.439514,0,0,0
2,2021-02-01 03:00:00,-29.095627,0,0,0
3,2021-02-01 04:00:00,-29.376926,0,0,0
4,2021-02-01 05:00:00,-27.988543,0,0,0


# 참고 자료

데이콘 대회

https://dacon.io/competitions/official/235720/overview/description

LSTM

https://dacon.io/competitions/official/235720/codeshare/2609

Prophet

https://dacon.io/competitions/official/235720/codeshare/2492